In [2]:
!pip3 install face_recognition

  ERROR: Could not find a version that satisfies the requirement widgets (from versions: none)
ERROR: No matching distribution found for widgets


In [1]:
import cv2
import ipywidgets as widgets
import face_recognition
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display, clear_output

alex_image = face_recognition.load_image_file("alex2.jpg")
alex_image_encoding = face_recognition.face_encodings(alex_image)[0]

# Put more photos here
# alex_image = face_recognition.load_image_file("alex2.jpg")
# alex_image_encoding = face_recognition.face_encodings(alex_image)[0]

known_face_encodings = [
    alex_image_encoding
#   List all encodings here    
#   alex_image_encoding
]
known_face_names = [
    "Alexander"
#   List all names here
#   "Alexander"
]
print('Learned encoding for', len(known_face_encodings), 'images.')

Learned encoding for 1 images.


In [2]:
def make_photo(self):
    global img_name
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("test")
    img_counter = 0
    while True:
        ret, frame = cam.read()
        cv2.imshow("test", frame)
        if not ret:
            break
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            img_name = "opencv_frame_{}.png".format(img_counter)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1
    cam.release()
    cv2.destroyAllWindows()

def check_photo(self):
    # Load an image with an unknown face
    unknown_image = face_recognition.load_image_file(img_name)

    # Find all the faces and face encodings in the unknown image
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    pil_image = Image.fromarray(unknown_image)
    draw = ImageDraw.Draw(pil_image)

    # Loop through each face found in the unknown image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
    display(pil_image)
     
    
 
button = widgets.Button(description='Make photo', button_style='info', tooltip='Start Game')
button_2 = widgets.Button(description='Who is that?', button_style='info', tooltip='Recognise')
button.on_click(make_photo)
button_2.on_click(check_photo)
down_box = widgets.HBox([button, button_2])
ui = widgets.VBox([down_box])
display(ui)